### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
school_data_complete.head(3)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
tot_num_schools = school_data_complete["School ID"].nunique()
tot_num_students = school_data_complete["Student ID"].count()
tot_budget = sum(school_data_complete["budget"].unique())
avg_math_score = school_data_complete["math_score"].mean()
avg_read_score = school_data_complete["reading_score"].mean()
perc_math_pass = sum(school_data_complete.math_score >= 70)/tot_num_students
perc_read_pass = sum(school_data_complete.reading_score >= 70)/tot_num_students
overall_pass = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)]['student_name'].count()/tot_num_students

In [4]:
district_summary_df = pd.DataFrame({"Total Schools": [tot_num_schools], 
                                    "Total Students": [tot_num_students], 
                                    "Total Budget": [tot_budget], 
                                    "Average Math Score": [avg_math_score], 
                                    "Average Reading Score": [avg_read_score], 
                                    "% Passing Math": [perc_math_pass], 
                                    "% Passing Reading": [perc_read_pass], 
                                    "% Overall Passing": [overall_pass]})

In [5]:
district_summary_df = district_summary_df.style.format({"Total Budget": "${:,.2f}",
                                   "Total Students": "{:,.0f}",
                                   "Average Reading Score": "{:.2f}", 
                                   "Average Math Score": "{:.2f}", 
                                   "% Passing Math": "{:.2%}", 
                                   "% Passing Reading": "{:.2%}", 
                                   "% Overall Passing": "{:.2%}"})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [6]:
school_summary = school_data_complete.groupby(["school_name"])

In [7]:
school_type = school_summary["type"].unique()
students_per_school = school_summary["Student ID"].count()
school_budget = school_summary["budget"].unique()
per_student_budget = school_budget / students_per_school
avg_math_school = school_summary["math_score"].mean()
avg_read_school = school_summary["reading_score"].mean()
math_pass_school = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/students_per_school
read_pass_school = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/students_per_school 
overall_pass_school = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)].groupby('school_name')['Student ID'].count()/students_per_school 

In [53]:
school_summary_df = pd.DataFrame({"School Type": school_type, 
                                    "Total Students": students_per_school, 
                                    "Total School Budget": school_budget,
                                    "Per Student Budget": per_student_budget,
                                    "Average Math Score": avg_math_school, 
                                    "Average Reading Score": avg_read_school, 
                                    "% Passing Math": math_pass_school, 
                                    "% Passing Reading": read_pass_school, 
                                    "% Overall Passing": overall_pass_school})
school_summary_df.index.name = None
school_style_df = school_summary_df.copy()

In [54]:
school_style_df['School Type'] = school_style_df['School Type'].astype(str).str[1:-1]
school_style_df['School Type'] = school_style_df['School Type'].astype(str).str[1:-1]
school_style_df['Total School Budget'] = school_style_df['Total School Budget'].astype(str).str[1:-1]
school_style_df['Per Student Budget'] = school_style_df['Per Student Budget'].astype(str).str[1:-1]
school_style_df.loc[:,"Total School Budget"] = school_style_df["Total School Budget"].astype(float).map("${:,.2f}".format)
school_style_df.loc[:,"Per Student Budget"] = school_style_df["Per Student Budget"].astype(float).map("${:,.2f}".format)
school_style_df = school_style_df.style.format({"Average Math Score": "{:.2f}", 
                                                    "Average Reading Score": "{:.2f}", 
                                                    "% Passing Math": "{:.2%}", 
                                                    "% Passing Reading": "{:.2%}", 
                                                    "% Overall Passing": "{:.2%}"})
school_style_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [63]:
school_summary_df.sort_values(by='% Overall Passing', ascending=False, inplace=True)
top_schools = school_summary_df.copy().head(5)

In [64]:
top_schools['School Type'] = top_schools['School Type'].astype(str).str[1:-1]
top_schools['School Type'] = top_schools['School Type'].astype(str).str[1:-1]
top_schools['Total School Budget'] = top_schools['Total School Budget'].astype(str).str[1:-1]
top_schools['Per Student Budget'] = top_schools['Per Student Budget'].astype(str).str[1:-1]
top_schools.loc[:,"Total School Budget"] = top_schools["Total School Budget"].astype(float).map("${:,.2f}".format)
top_schools.loc[:,"Per Student Budget"] = top_schools["Per Student Budget"].astype(float).map("${:,.2f}".format)
top_schools = top_schools.style.format({"Average Math Score": "{:.2f}", 
                                                    "Average Reading Score": "{:.2f}", 
                                                    "% Passing Math": "{:.2%}", 
                                                    "% Passing Reading": "{:.2%}", 
                                                    "% Overall Passing": "{:.2%}"})
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [70]:
school_summary_df.sort_values(by='% Overall Passing', ascending=True, inplace=True)
bottom_schools = school_summary_df.copy().head(5)

In [71]:
bottom_schools['School Type'] = bottom_schools['School Type'].astype(str).str[1:-1]
bottom_schools['School Type'] = bottom_schools['School Type'].astype(str).str[1:-1]
bottom_schools['Total School Budget'] = bottom_schools['Total School Budget'].astype(str).str[1:-1]
bottom_schools['Per Student Budget'] = bottom_schools['Per Student Budget'].astype(str).str[1:-1]
bottom_schools.loc[:,"Total School Budget"] = bottom_schools["Total School Budget"].astype(float).map("${:,.2f}".format)
bottom_schools.loc[:,"Per Student Budget"] = bottom_schools["Per Student Budget"].astype(float).map("${:,.2f}".format)
bottom_schools = bottom_schools.style.format({"Average Math Score": "{:.2f}", 
                                                    "Average Reading Score": "{:.2f}", 
                                                    "% Passing Math": "{:.2%}", 
                                                    "% Passing Reading": "{:.2%}", 
                                                    "% Overall Passing": "{:.2%}"})
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
